In [7]:
import gzip
import os
from subprocess import call
from os.path import join, exists

In [8]:
files = [
    "Triticum_aestivum.IWGSC.dna.toplevel.fa.gz",
#     "Zea_mays.B73_RefGen_v4.dna.toplevel.fa.gz",
#     "Oryza_sativa.IRGSP-1.0.dna.toplevel.fa.gz"
#     "Arabidopsis_thaliana.TAIR10.dna.toplevel.fa.gz",
#     "Solanum_lycopersicum.SL3.0.dna.toplevel.fa.gz",
#     "Gossypium_raimondii.Graimondii2_0.dna.toplevel.fa.gz",
#     "Solanum_tuberosum.SolTub_3.0.dna.toplevel.fa.gz",
]
assembly_out = "assemblies"

In [9]:
# Parse chromosome names
parsed_files = []
for file in files:
    do_skip = False
    print("Parsing {} ...".format(file))
    basefile = ".".join(os.path.splitext(file)[0].split(".")[0:-3]) + ".fa"
    parsed_files.append(basefile)
    if not exists(basefile):
        with gzip.open(join(assembly_out, file), 'rb') as f, open(join(assembly_out, basefile), "w") as fw:
            for line in f:
                line = line.decode('utf-8')
                if line.startswith(">"):
                    fields = line.strip().split()
                    contig = fields[0].split(">")[1].strip()
                    if "dna:chromosome" in line:
                        do_skip = False
                    elif "dna:contig" in line:
                        do_skip = True
                    elif "dna:supercontig" in line:
                        do_skip = True
                    elif "dna:scaffold" in line:
                        do_skip = True
                    else:
                        print(line)
                    contig = ">{}\n".format(contig)
                    fw.write(contig)
                else:
                    if not do_skip:
                        fw.write(line)
    print("Parsing {} ... done".format(file))


Parsing Triticum_aestivum.IWGSC.dna.toplevel.fa.gz ...
Parsing Triticum_aestivum.IWGSC.dna.toplevel.fa.gz ... done


In [10]:
# FASTA -> BLASTDB
for file in parsed_files:
    cmd = [
        "makeblastdb",
        "-dbtype nucl",
        "-in {}".format(join(assembly_out, file)),
        "-out {}".format(join(assembly_out, ".".join(file.split(".")[0:-1]))),
        "-parse_seqids",
        "-max_file_sz 4G"
    ]
    cmd = " ".join(cmd)
    print(cmd)
    call(cmd, shell=True)

makeblastdb -dbtype nucl -in assemblies/Triticum_aestivum.IWGSC.fa -out assemblies/Triticum_aestivum.IWGSC -parse_seqids -max_file_sz 4G
